In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Base URL của trang công thức
base_url = "https://nutritionfacts.org/recipes/"
response = requests.get(base_url)
soup = BeautifulSoup(response.content, 'html.parser')

# Lấy các link công thức
recipe_links = soup.select("a.wpupg-type-wprm_recipe")
recipe_urls = [link.get('href') for link in recipe_links]

# Khởi tạo dictionary lưu dữ liệu
data = {
    "Recipe Name": [],
    "Ingredient Group": [],
    "Ingredient Name": [],
    "Amount": [],
    "Unit": []
}

# Duyệt qua từng công thức và thu thập dữ liệu
for recipe_url in recipe_urls:
    response = requests.get(recipe_url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Lấy tên công thức
    recipe_name = soup.select_one("h1").get_text(strip=True)

    # Lấy các nhóm nguyên liệu, có thể có thẻ h4 hoặc không
    ingredient_groups = soup.select("div.wprm-recipe-ingredient-group")

    for group in ingredient_groups:
        group_name = group.select_one("h4")
        ingredient_group = group_name.get_text(strip=True) if group_name else ""  # Lấy tên nhóm nếu có

        # Duyệt qua từng nguyên liệu trong nhóm
        ingredients = group.select("li.wprm-recipe-ingredient")

        for ingredient in ingredients:
            amount = ingredient.select_one("span.wprm-recipe-ingredient-amount")
            unit = ingredient.select_one("span.wprm-recipe-ingredient-unit")
            name = ingredient.select_one("span.wprm-recipe-ingredient-name")

            # Lấy dữ liệu nếu có, nếu không thì bỏ trống
            ingredient_amount = amount.get_text(strip=True) if amount else ""
            ingredient_unit = unit.get_text(strip=True) if unit else ""
            ingredient_name = name.get_text(strip=True) if name else ""

            # Lưu dữ liệu vào bảng
            data["Recipe Name"].append(recipe_name)
            data["Ingredient Group"].append(ingredient_group)
            data["Ingredient Name"].append(ingredient_name)
            data["Amount"].append(ingredient_amount)
            data["Unit"].append(ingredient_unit)

    print(f"Thu thập xong nguyên liệu cho công thức: {recipe_name}")

# Tạo DataFrame từ dictionary
df = pd.DataFrame(data)

# Lưu dữ liệu vào file Excel
df.to_excel("recipes_ingredients_grouped.xlsx", index=False)

print("Xuất dữ liệu ra file Excel thành công: recipes_ingredients_grouped.xlsx")

Thu thập xong nguyên liệu cho công thức: Kale Pesto Basil Wraps
Thu thập xong nguyên liệu cho công thức: Southwest Kale Salad with Cumin-Tomato Dressing
Thu thập xong nguyên liệu cho công thức: Chickpea Chili
Thu thập xong nguyên liệu cho công thức: Jicama Mango Salad
Thu thập xong nguyên liệu cho công thức: Quinoa and Vegetable Stew
Thu thập xong nguyên liệu cho công thức: Ultimate Sauce
Thu thập xong nguyên liệu cho công thức: Cauliflower Bisque
Thu thập xong nguyên liệu cho công thức: Cinnamon Roll Oatmeal
Thu thập xong nguyên liệu cho công thức: Rainbow Root Veggie Stew
Thu thập xong nguyên liệu cho công thức: Lentil–Quinoa Tacos
Thu thập xong nguyên liệu cho công thức: Sweet Potato Mac N' Cheese
Thu thập xong nguyên liệu cho công thức: Sweet and Smoky Dressing
Thu thập xong nguyên liệu cho công thức: Plant-Based Zucchini Lasagna
Thu thập xong nguyên liệu cho công thức: Chocolate–Cherry Nice Cream
Thu thập xong nguyên liệu cho công thức: Peach and Papaya Salad
Thu thập xong nguyên 

In [ ]:
file_path = 'recipes_ingredients_grouped.xlsx'

# Từ khóa cho từng loại đặc điểm
PROCESSED_KEYWORDS = [
    'cooked', 'roasted', 'juiced', 'baking', 'can', 'canned', 'BPA', 'vinegar', 'salt-free',
    'Savory Spice Blend', 'tortillas', 'powder', 'butter', 'tahini', 'tofu', 'BPA-free',
    'Tetra Pak', 'Salsa', 'Super-Charged Spice Blend', 'sauce', 'flakes', 'unsweetened',
    'miso', 'broth', 'flour', 'tempeh', 'syrup', 'blackstrap molasses', 'Nutty Parm', 'BROL',
    'refried', 'guacamole', 'cashew cream', 'milk', 'cornmeal', 'bread crumbs', 'ranch dressing',
    'balsamic date glaze', 'cacao nibs', 'blended', 'chlorella', 'powder', 'ground', 'nutritional yeast',
    'mustard', 'paprika', 'extract'
]
PRELIMINARY_PROCESSED_KEYWORDS = [
    'chopped', 'cut', 'stems removed', 'shredded', 'diced', 'washed', 'de-stemmed', 'sliced',
    'torn', 'grated', 'cubed', 'seeded', 'quartered', 'minced', 'peeled', 'julienned', 'halved',
    'cored', 'soaked', 'pitted', 'meat only', 'hulled', 'slivered', 'split', 'scraped', 'rinsed',
    'drained', 'picked', 'crushed', 'smashed', 'squeezed dry'
]
FRESH_KEYWORDS = ['fresh']
FROZEN_KEYWORDS = ['frozen', 'thawed frozen', 'freeze dried', 'shelled frozen']
DRIED_KEYWORDS = ['dried', 'dry']
OPTIONAL_KEYWORDS = ['optional', 'if desired', 'as desired', 'if needed', 'as needed']
GARNISH_KEYWORDS = ['garnish']
FOR_DIPPING_KEYWORDS = ['dipping']
TO_TASTE_KEYWORDS = ['taste']
DIFFERENT_CHOICES_KEYWORDS = ['other', 'choice', 'choices', 'kind']

# Hàm để tìm vị trí của từ khóa đầu tiên xuất hiện trong chuỗi
def find_first_keyword(ingredient, keywords):
    pattern = '|'.join([re.escape(keyword) for keyword in keywords])
    match = re.search(pattern, ingredient)
    if match:
        return match.start()  # Trả về vị trí của từ khóa
    return float('inf')  # Nếu không tìm thấy, trả về giá trị lớn vô hạn

# Hàm trích xuất đặc điểm từ ingredient name
def extract_ingredient_features(ingredient):
    features = {
        'ingredient': None,
        'processed': False,
        'preliminary_processed': False,
        'fresh': False,
        'frozen': False,
        'dried': False,
        'optional': False,
        'garnish': False,
        'for_dipping': False,
        'to_taste': False,
        'different_choices': False
    }

    # Chuẩn hóa tên nguyên liệu
    ingredient_lower = ingredient.lower()

    # Tìm vị trí của từ khóa đầu tiên trong processed và preliminary_processed
    processed_pos = find_first_keyword(ingredient_lower, PROCESSED_KEYWORDS)
    preliminary_pos = find_first_keyword(ingredient_lower, PRELIMINARY_PROCESSED_KEYWORDS)

    # Nếu processed xuất hiện trước preliminary_processed, đặt processed = True và ngược lại
    if processed_pos < preliminary_pos:
        features['processed'] = True
    elif preliminary_pos < processed_pos:
        features['preliminary_processed'] = True

    # Kiểm tra các từ khóa khác
    features['fresh'] = any(keyword.lower() in ingredient_lower for keyword in FRESH_KEYWORDS)
    features['frozen'] = any(keyword.lower() in ingredient_lower for keyword in FROZEN_KEYWORDS)
    features['dried'] = any(keyword.lower() in ingredient_lower for keyword in DRIED_KEYWORDS)
    features['optional'] = any(keyword.lower() in ingredient_lower for keyword in OPTIONAL_KEYWORDS)
    features['garnish'] = any(keyword.lower() in ingredient_lower for keyword in GARNISH_KEYWORDS)
    features['for_dipping'] = any(keyword.lower() in ingredient_lower for keyword in FOR_DIPPING_KEYWORDS)
    features['to_taste'] = any(keyword.lower() in ingredient_lower for keyword in TO_TASTE_KEYWORDS)
    features['different_choices'] = any(keyword.lower() in ingredient_lower for keyword in DIFFERENT_CHOICES_KEYWORDS)

    # Loại bỏ các mô tả và chỉ giữ lại tên nguyên liệu
    clean_ingredient = re.sub(r'\(.*?\)|,.*', '', ingredient).strip()
    features['ingredient'] = clean_ingredient

    return features

# Đọc file Excel và áp dụng trích xuất đặc điểm
df1 = pd.read_excel(file_path)
df1_features = df['Ingredient Name'].apply(lambda x: pd.Series(extract_ingredient_features(x)))

# Kết hợp dữ liệu đặc điểm với dữ liệu gốc
df1 = pd.concat([df1, df1_features], axis=1)

df1['Ingredient Group'] = df['Ingredient Group'].str.replace(':', '', regex=False)

df1.to_excel('recipes_ingredients_grouped_full.xlsx')

In [ ]:
df1.head(55)

,Recipe Name,Ingredient Group,Ingredient Name,Amount,Unit,ingredient,processed,preliminary_processed,fresh,frozen,dried,optional,garnish,for_dipping,to_taste,different_choices
0,Kale Pesto Basil Wraps,Green Wrap,"(100g) chopped kale, stems removed",4,cups,chopped kale,False,True,False,False,False,False,False,False,False,False
1,Kale Pesto Basil Wraps,Green Wrap,(50g) fresh basil leaves,2,cups,fresh basil leaves,False,False,True,False,False,False,False,False,False,False
2,Kale Pesto Basil Wraps,Green Wrap,"jalapeños (about 28g), seeds removed",2,NaN,jalapeños,False,False,False,False,False,False,False,False,False,False
3,Kale Pesto Basil Wraps,Green Wrap,(170g) chopped zucchini,1,cup,chopped zucchini,False,True,False,False,False,False,False,False,False,False
4,Kale Pesto Basil Wraps,Green Wrap,celery ribs (about 80g),2,NaN,celery ribs,False,False,False,False,False,False,False,False,False,False
5,Kale Pesto Basil Wraps,Green Wrap,pitted medjool dates (about 75g),3,NaN,pitted medjool dates,False,True,False,False,False,False,False,False,False,False
6,Kale Pesto Basil Wraps,Green Wrap,hemp seeds,2,tablespoons,hemp seeds,False,False,False,False,False,False,False,False,False,False
7,Kale Pesto Basil Wraps,Green Wrap,garlic cloves,3,NaN,garlic cloves,False,False,False,False,False,False,False,False,False,False
8,Kale Pesto Basil Wraps,Green Wrap,lemon juice,2,tablespoons,lemon juice,False,False,False,False,False,False,False,False,False,False
9,Kale Pesto Basil Wraps,Green Wrap,nutritional yeast,2,tablespoons,nutritional yeast,True,False,False,False,False,False,False,False,False,False


In [ ]:
"Sweet Potato Mac N' Cheese": {
    'No Group': {
        'Step 1': {
            'step': 'Roast the sweet potato and garlic at 400°F (205°C) for 25 minutes.',
            'ingredients': ['4 nan garlic cloves',]
        },
        'Step 2': {
            'step': 'In a high-speed blender, combine the roasted sweet potato, roasted garlic, soy milk, and nutritional yeast and process until smooth.',
            'ingredients': [
                '4 nan garlic cloves',
                '1 cup unsweetened soy milk',
                '¼ cup nutritional yeast'
            ]
        },
        'Step 3': {
            'step': 'In a saucepan, pour the sweet potato sauce over cooked pasta. Cook on medium heat until the sauce is warm.',
            'ingredients': ['8 oz 100% whole-grain pasta, cooked']
        },
        ...
    }
}
